### Evaluación de Modelos de Regresión

En esta sección, se evalúan varios modelos de regresión para predecir la "Happiness Score" utilizando un conjunto de características. Los modelos incluyen regresión lineal, regresión Ridge, Lasso, ElasticNet, K-Nearest Neighbors (KNN), Random Forest y XGBoost. Se realiza la codificación One-Hot para la columna "Country" y luego se divide el conjunto de datos en entrenamiento y prueba utilizando `train_test_split`. Posteriormente, se entrena y evalúa cada modelo en función de su desempeño, medido por el R2 Score, para determinar cuál es el mejor modelo.

### Proceso:
1. **Codificación One-Hot**: Se convierte la columna "Country" en variables dummy (0 o 1), eliminando la primera categoría para evitar la multicolinealidad.
2. **Separación de características y variable objetivo**: Se separan las columnas de las características (`X`) y la variable objetivo (`y`), que es "Happiness Score".
3. **División de datos**: El conjunto de datos se divide en entrenamiento (70%) y prueba (30%) utilizando `train_test_split`.
4. **Evaluación de los modelos**: Se entrenan y evalúan los siguientes modelos:
   - **Linear Regression**
   - **Ridge Regression**
   - **Lasso Regression**
   - **ElasticNet Regression**
   - **K-Nearest Neighbors Regression**
   - **Random Forest Regression**
   - **XGBoost Regression**
5. **Cálculo del R2 Score**: Para cada modelo, se calcula el R2 Score en el conjunto de prueba para medir su capacidad predictiva.
6. **Selección del mejor modelo**: Se identifica el modelo con el mejor R2 Score.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# Cargar los datos
file_path = '../data/Model_Data.csv'
data = pd.read_csv(file_path)

# Realizar codificación one-hot para la columna 'Country'
data = pd.get_dummies(data, columns=['Country'], drop_first=True)

# Separar las características (X) y la variable objetivo (y)
X = data.drop(columns=['Happiness Score'])
y = data['Happiness Score']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir los modelos a evaluar
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Elastic Net Regression": ElasticNet(alpha=0.1, l1_ratio=0.5),
    "K-Nearest Neighbors Regression": KNeighborsRegressor(n_neighbors=5),
    "Random Forest Regression": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost Regression": XGBRegressor(n_estimators=100, random_state=42)
}

# Evaluación de modelos
results = {}
best_model_name = None
best_model = None
best_r2_score = float('-inf')

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    results[model_name] = {"R2 Score": r2}
    
    # Verificar si este modelo tiene el mejor R2 score
    if r2 > best_r2_score:
        best_r2_score = r2
        best_model = model
        best_model_name = model_name

# Imprimir los resultados de R2 para cada modelo
for model_name, metrics in results.items():
    print(f"{model_name}: R2 Score = {metrics['R2 Score']:.4f}")

print(f"\nBest model: {best_model_name} with R2 Score = {best_r2_score:.4f}")


Linear Regression: R2 Score = 0.9392
Ridge Regression: R2 Score = 0.9280
Lasso Regression: R2 Score = 0.6092
Elastic Net Regression: R2 Score = 0.6485
K-Nearest Neighbors Regression: R2 Score = 0.8527
Random Forest Regression: R2 Score = 0.8538
XGBoost Regression: R2 Score = 0.8758

Best model: Linear Regression with R2 Score = 0.9392


# Evaluación de Modelos de Regresión

Este código entrena y evalúa varios modelos de regresión utilizando un conjunto de datos, preprocesando las características adecuadamente, y selecciona el mejor modelo basado en el R² Score.


In [20]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Cargar los datos
data = pd.read_csv('../data/Model_Data.csv')

# Definir las características y la variable objetivo
X = data.drop(columns=['Happiness Score'])
y = data['Happiness Score']

# Preprocesar los datos: aplicar One-Hot Encoding a la columna 'Country' y escalar el resto
preprocessor = ColumnTransformer(
    transformers=[
        ('country', OneHotEncoder(handle_unknown='ignore'), ['Country']),  # Codificar la columna 'Country' y manejar categorías desconocidas
        ('num', SimpleImputer(strategy='mean'), X.columns.difference(['Country', 'Year']))  # Imputar valores faltantes para otras columnas numéricas
    ])

# Crear un diccionario de modelos
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'Support Vector Regression': SVR(),
    'K-Nearest Neighbors': KNeighborsRegressor()
}

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Variables para almacenar el mejor modelo y su rendimiento
best_model = None
best_r2 = float('-inf')  # Inicializamos con el peor valor posible para R^2
best_model_name = ""  # Para almacenar el nombre del mejor modelo

# Entrenar y evaluar cada modelo
for model_name, model in models.items():
    # Crear un pipeline para cada modelo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # Entrenar el modelo
    pipeline.fit(X_train, y_train)
    
    # Hacer predicciones
    y_pred = pipeline.predict(X_test)
    
    # Evaluar el modelo
    r2 = r2_score(y_test, y_pred)
    
    print(f'\nModelo: {model_name}')
    print(f'R^2 Score: {r2}')
    
    # Si el modelo actual es el mejor, lo guardamos
    if r2 > best_r2:
        best_r2 = r2
        best_model = pipeline
        best_model_name = model_name

print(f"\nBest model: {best_model_name} with R2 Score = {best_r2:.4f}")




Modelo: Linear Regression
R^2 Score: 0.9495858523063235

Modelo: Random Forest
R^2 Score: 0.8486986222615859

Modelo: Gradient Boosting
R^2 Score: 0.842152498124545

Modelo: AdaBoost
R^2 Score: 0.7726456924117253

Modelo: Support Vector Regression
R^2 Score: 0.9237282170578003

Modelo: K-Nearest Neighbors
R^2 Score: 0.8937572721799011

Best model: Linear Regression with R2 Score = 0.9496


## Guardar el Mejor Modelo en un Archivo `.pkl`

Una vez que se ha entrenado y evaluado el modelo, es posible guardar el mejor modelo en un archivo `.pkl` para poder usarlo más adelante sin necesidad de volver a entrenarlo.


In [21]:
# Guardar el mejor modelo en un archivo .pkl
with open('../Model/Regression_Model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Modelo guardado exitosamente en 'Model/Regression_Model'")

Modelo guardado exitosamente en 'Model/Regression_Model'
